In [ ]:
import numpy as np
from numpy import pi
from pyquil.api import get_qc
from pyquil.api._devices import get_lattice
from pyquil import Program
from pyquil.gates import *

import matplotlib.pyplot as plt
%matplotlib inline



# Specify lattice name and show any stored specs

In [ ]:
# lattice_name = 'Aspen-1-5Q-B'
lattice_name = '9q-square-noisy-qvm'
# lattice = get_lattice(lattice_name)
# stored_specs = lattice.get_specs()
# print(stored_specs)

# Create qc object, get qubits, and display the topology

# BE SURE TO SET as_qvm TRUE or FALSE as desired!!!

In [ ]:
import networkx as nx

qc = get_qc(lattice_name, as_qvm=True, noisy=True)
qubits = qc.qubits()
print(qubits)
graph = qc.qubit_topology()
nx.draw_networkx(graph, with_labels=True)

# Active Reset Error

In [ ]:
from forest_benchmarking.readout import estimate_joint_reset_confusion
single_qubit_reset_cms = estimate_joint_reset_confusion(qc, qubits, num_trials = 10, joint_group_size = 1,
                                   use_active_reset = True)


Confusion matrix, Avg Fidelity

In [ ]:
print(single_qubit_reset_cms)
print([np.round(np.sum(cm, axis=0)[0]/2, 3) for cm in single_qubit_reset_cms.values()])


# Readout Errors

In [ ]:
from forest_benchmarking.readout import estimate_joint_confusion_in_set, marginalize_confusion_matrix
single_qubit_cms = estimate_joint_confusion_in_set(qc, qubits, num_shots=5000, joint_group_size=1,
                                    use_param_program=True, use_active_reset=False)


Confusion matrix, Avg Fidelity, Asymmetry

In [ ]:
print(single_qubit_cms)
print([np.round(np.trace(cm)/2, 3) for cm in single_qubit_cms.values()])
from forest_benchmarking.utils import sigma_z
print([np.round(np.trace(sigma_z @ cm)/2, 3) for cm in single_qubit_cms.values()])

Simultaneous Confusion Matrix (pairwise; can try len(qubits) but may be too slow)

In [ ]:
pairwise_cms = estimate_joint_confusion_in_set(qc, qubits, num_shots=1000, joint_group_size=2,
                                    use_param_program=True, use_active_reset=False)

 Look for Significant Correlated Error

In [ ]:
marginal_absolute_tolerance = .02 # determines acceptable level of correlation

for qubit_pair, pair_cm in pairwise_cms.items():
    marginal_one_qs = [(qubit, marginalize_confusion_matrix(pair_cm, qubit_pair, [qubit])) for qubit in qubit_pair]
    for qubit, marginal_cm in marginal_one_qs:
        if not np.allclose(single_qubit_cms[(qubit,)], marginal_cm, atol=marginal_absolute_tolerance):
            print("Q" + str(qubit) + " readout is different when measuring pair", qubit_pair)
            
joint_absolute_tolerance = .03
for qubit_pair, pair_cm in pairwise_cms.items():
    joint_single_q_cm = np.kron(single_qubit_cms[(qubit_pair[0],)], single_qubit_cms[(qubit_pair[1],)])
    if not np.allclose(joint_single_q_cm, pair_cm, atol=joint_absolute_tolerance):
        print(qubit_pair, "exhibits correlated readout error")


# T1/T2

Neither estimation of T1 or T2 will work on a QVM

## T1

In [ ]:
from forest_benchmarking.qubit_spectroscopy import run_t1, exponential_decay_curve, fit_to_exponential_decay_curve
MICROSECOND = 1e-6
def get_T1s_from_data_fit(df, show_plot: bool = False,
                                                  filename: str = None) -> np.ndarray:
    """
    Plot T1 experimental data and fitted exponential decay curve.

    :param df: Experimental results to plot and fit exponential decay curve to.
    :return: list of estimated t1s in microseconds
    """
    t1s = np.array([])
    for q in df['qubit'].unique():
        df2 = df[df['qubit'] == q].sort_values('time')
        x_data = df2['time']
        y_data = df2['avg']

        plt.plot(x_data, y_data, 'o-', label=f"QC{q} T1 data")

        try:
            fit_params, fit_params_errs = fit_to_exponential_decay_curve(x_data, y_data)
        except RuntimeError:
            print(f"Could not fit to experimental data for QC{q}")
            t1s = np.append(t1s,[np.nan])
        else:
            plt.plot(x_data, exponential_decay_curve(x_data, *fit_params),
                     label=f"QC{q} fit: T1={fit_params[1] / MICROSECOND:.2f}us")
            t1s = np.append(t1s,[fit_params[1]/MICROSECOND])

    plt.xlabel("Time [s]")
    plt.ylabel("Excited state visibility")
    plt.title("T1 decay")

    plt.legend(loc='best')
    plt.tight_layout()
    if filename is not None:
        plt.savefig(filename)
        
    if show_plot:
        plt.show()
    else:
        plt.close()
        
    return t1s
    
t1_df = run_t1(qc, qubits, stop_time = 60*MICROSECOND, n_shots = 1000, num_points = 15)
t1s = get_T1s_from_data_fit(t1_df)
print("T1s in microseconds:", np.round(t1s,1))

# $T_2^*$ Ramsey

In [ ]:
from forest_benchmarking.qubit_spectroscopy import run_t2, exponentially_decaying_sinusoidal_curve, fit_to_exponentially_decaying_sinusoidal_curve
MHZ = 1e6
MICROSECOND = 1e-6
from typing import List
def get_T2s_from_data_fit(df, show_plot: bool = False, filename: str = None,
                                                detuning: float = 5e6) -> np.ndarray:
    """
    Plot T2 experimental data and fitted exponential decay curve.

    :param df: Experimental results to plot and fit exponential decay curve to.
    :param detuning: Detuning frequency used in experiment creation.
    :return: list of estimated t2s
    """
    t2s=np.array([])
    detunings=np.array([])
    for q in df['qubit'].unique():
        df2 = df[df['qubit'] == q].sort_values('time')
        x_data = df2['time']
        y_data = df2['avg']

        plt.plot(x_data, y_data, 'o-', label=f"QC{q} T2 data")

        try:
            fit_params, fit_params_errs = fit_to_exponentially_decaying_sinusoidal_curve(x_data,
                                                                                         y_data,
                                                                                         detuning)
        except RuntimeError:
            print(f"Could not fit to experimental data for QC {q}")
            t2s = np.append(t2s,[np.nan])
            detunings = np.append(detunings, [np.nan])
        else:
            plt.plot(x_data, exponentially_decaying_sinusoidal_curve(x_data, *fit_params),
                     label=f"QC{q} fit: freq={fit_params[2] / MHZ:.2f}MHz, "
                           f"T2={fit_params[1] / MICROSECOND:.2f}us")
            t2s = np.append(t2s,[fit_params[1]/MICROSECOND])
            detunings = np.append(detunings, fit_params[2] / MHZ)
            
    plt.xlabel("Time [s]")
    plt.ylabel("Excited state visibility")
    plt.title("T2 Ramsey decay")

    plt.legend(loc='best')
    plt.tight_layout()
    if filename is not None:
        plt.savefig(filename)

    if show_plot:
        plt.show()
    else:
        plt.close()
        
    return t2s, detunings
    
t2_df,detuning_used = run_t2(qc, qubits, stop_time = 10*MICROSECOND, n_shots = 1000, num_points = 50)
t2s, detunings = get_T2s_from_data_fit(t2_df, show_plot=True, detuning=detuning_used)
print("T2s in microseconds:", np.round(t2s,1))
print("Detunings in MHz:", np.round(detunings,1))

# Single Qubit Gate Fidelity

In [ ]:
from pyquil.api import get_benchmarker
from forest_benchmarking.rb import (add_sequences_to_dataframe,
                            add_survivals,
                            generate_simultaneous_rb_sequence,
                            rb_dataframe,
                            run_rb_measurement,
                            survivals_by_qubits,
                           fit_standard_rb)
from forest_benchmarking.analysis.fitting import make_figure
bm = get_benchmarker()

Estimate 1q fidelity separately 

In [ ]:
separate_1q_dfs = []
depths_1q, survivals_1q, survival_errs_1q = {}, {}, {}
separate_1q_rb_decays = []

for qubit in qubits:
    df = rb_dataframe(rb_type="std-1q",
                  subgraph=[(qubit,)],
                  depths=3 * 2 ** np.arange(4, dtype=np.uint8),
                  num_sequences=50)
    df = add_sequences_to_dataframe(df, bm)
    df = run_rb_measurement(df, qc, num_trials=500)
    df = add_survivals(df)
    separate_1q_dfs.append(df)
    
    depths_1q[qubit], survivals_1q[qubit], survival_errs_1q[qubit] = survivals_by_qubits(df, (qubit,)) 
    fit = fit_standard_rb(depths_1q[qubit], survivals_1q[qubit], weights=1/survival_errs_1q[qubit])
    separate_1q_rb_decays.append(fit.params['decay'].value)
    
    fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Survival Probability")
    
print(separate_1q_rb_decays)

Estimate simultaneous 1q fidelity

In [ ]:
separate_1q_dfs = []
depths_1q, survivals_1q, survival_errs_1q = {}, {}, {}
simult_1q_rb_decays = []

simult_1q_df = rb_dataframe(rb_type="sim-1q",
                  subgraph=[(qubit,) for qubit in qubits],
                  depths=3 * 2 ** np.arange(4, dtype=np.uint8),
                  num_sequences=50)
simult_1q_df = add_sequences_to_dataframe(simult_1q_df, bm)
simult_1q_df = run_rb_measurement(simult_1q_df, qc, num_trials=500)
simult_1q_df = add_survivals(simult_1q_df)

for qubit in qubits:
    depths, survivals, survival_errs = {}, {}, {}
    depths[qubit], survivals[qubit], survival_errs[qubit] = survivals_by_qubits(simult_1q_df, (qubit,))
    fit = fit_standard_rb(depths[qubit], survivals[qubit], weights=1/survival_errs[qubit])
    simult_1q_rb_decays.append(fit.params['decay'].value)
    
    fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Survival Probability")
    
print(simult_1q_rb_decays)

In [ ]:
for qubit, (simult, sep) in zip(qubits, zip(simult_1q_rb_decays, separate_1q_rb_decays)):
    if not np.allclose(simult, sep, atol = .05):
        print("qubit " + str(qubit) + " may be suffering from significant 1q cross-talk.")

# CZ Fidelity (SLOW on qvm. Set variance as desired)

In [ ]:
from forest_benchmarking.dfe import generate_process_dfe_experiment, acquire_dfe_data, direct_fidelity_estimate

desired_variance = .1

if not bm:
    bm = get_benchmarker()

cz_fidelities = []
for edge in graph.edges():
    p = Program(CZ(edge[0], edge[1]))
    process_exp = generate_process_dfe_experiment(p, bm)
    data,cal = acquire_dfe_data(process_exp, qc, desired_variance)
    fidelity_est = direct_fidelity_estimate(data,cal,'process')
    print(edge, " : ", fidelity_est.fid_point_est, "+/-", np.sqrt(fidelity_est.fid_var_est))
    cz_fidelities.append((edge, fidelity_est))


# CZ Cross Talk

In [ ]:
from forest_benchmarking.rpe import (generate_rpe_experiments, 
                             acquire_rpe_data, 
                             find_expectation_values, 
                             robust_phase_estimate)
CZ_qubits = (0,1)
measure_qubit = 3

num_depths = 6
multiplicative_factor = 100

rpe_experiments = generate_rpe_experiments(Program(CZ(*CZ_qubits)), 
                                       num_depths, 
                                       measurement_qubit = measure_qubit)
rpe_experiments = acquire_rpe_data(rpe_experiments, qc, multiplicative_factor)
xs, ys, x_stds, y_stds = find_expectation_values(rpe_experiments)

rz_local_phase_on_qubit = robust_phase_estimate(xs, ys, x_stds, y_stds)
print(rz_local_phase_on_qubit)

# All qubits RX calibration

In [ ]:
angles = [-pi, -pi/2, pi/2, pi]

num_depths = 6 # max depth of 2^(num_depths - 1)
factor = 100 # Multiply the optimal number of shots by this factor for each experiment
add_error = None # Try to correct for this additivie error by increasing the number of shots in an optimal way

for qubit in qubits:
    print("Q" + str(qubit))
    for angle in angles:
        experiments_1q = generate_rpe_experiments(RX(angle, qubit), num_depths, axis=(pi/2,0))
        experiments_1q = acquire_rpe_data(experiments_1q, qc, multiplicative_factor = factor, additive_error = add_error)
        xs, ys, x_stds, y_stds = find_expectation_values(experiments_1q)
        estimated_angle = robust_phase_estimate(xs, ys, x_stds, y_stds)
        if angle < 0 and estimated_angle > 0:
            estimated_angle = estimated_angle - 2*pi
        if angle > 0 and estimated_angle < 0:
            estimated_angle = estimated_angle + 2*pi
        print("Est. Angle:  ", np.round(estimated_angle, 4))
        print("Ideal Angle: ", np.round(angle, 4))
        print()
